In [2]:
!pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/50/28/fdfe9711d553cfaf217cafc9b8ba21479c663b6504cc96652465b153e04b/streamlit-1.26.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/2b/40/ff33821bca16cac30f8d9c3244ac961416f40bf2d3261a1250aabda33a6f/altair-5.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/a9/c9/c8a7710f2cedcb1db9224fdd4d8307c9e48cbddc46c18b515fefc0f1abbe/cachetools-5.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<5,>=3.20 from https://files.pythonhosted.org/packages/14/ff/10f746c03212fe48576b2c0f5ada73c3400b6d90f769728c4f07656d8b27/protobuf-4.24.2-cp310-abi3-win_amd64.whl.metadata
     ---------------------------------------- 0.0/164.8 kB ? eta -:--:--
     -------------------------------------- 164.8/164.8 kB 9.7 MB/s eta 0:00:00
  

In [48]:
!pip install notebook

In [5]:
!pip install geopandas

  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/cb/7b/2fb6ab376c78d2e60812ad0bf4b24889309d2aeb543f54efcd586c69907f/geopandas-0.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for fiona>=1.8.19 from https://files.pythonhosted.org/packages/b0/7f/2de46a2630f609b7520d74ffc7692d4969b1fa1dd3c82f62c7967183d365/Fiona-1.9.4.post1-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/50.6 kB ? eta -:--:--
     ---------------------------------------- 50.6/50.6 kB 1.3 MB/s eta 0:00:00
  Obtaining dependency information for pyproj>=3.0.1 from https://files.pythonhosted.org/packages/c8/5a/215a1894e50167d91b471d8fc413ca30034c48e5d3dfac78d12df4c840d5/pyproj-3.6.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     ------------- -------------------------- 0.5/1.4 MB 9.6 MB/s eta 0:00:01
     --------------------------------- ------ 1.1/1.4 MB 11.9 MB

In [10]:
!pip install geopy

  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e1/58/9289c6a03116025cdb61461d99b2493daa4967a80b13755463d71a0affeb/geopy-2.4.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
     ---------------------------------------- 40.3/40.3 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------------------------- 125.4/125.4 kB ? eta 0:00:00


In [17]:
!pip install folium

     ---------------------------------------- 0.0/102.3 kB ? eta -:--:--
     -------------------------------------- 102.3/102.3 kB 6.1 MB/s eta 0:00:00


In [21]:
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
import time
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
import streamlit as st
import folium

In [22]:
info_df= pd.read_excel('data.xlsx')

In [23]:
info_df

,Ticker,Name,Sector,MarketCap,Headquarters,Latitude,Longitude,District
0,GOOGL,Alphabet Inc.,Communication Services,1.642359e+12,"1600 Amphitheatre Parkway, Mountain View, CA, ...",37.421764,-122.084614,NaN
1,GOOG,Alphabet Inc.,Communication Services,1.642355e+12,"1600 Amphitheatre Parkway, Mountain View, CA, ...",37.421764,-122.084614,NaN
2,AMZN,"Amazon.com, Inc.",Consumer Cyclical,1.374950e+12,"410 Terry Avenue North, Seattle, WA, United St...",47.622297,-122.336500,NaN
3,NVDA,NVIDIA Corporation,Technology,1.136645e+12,"2788 San Tomas Expressway, Santa Clara, CA, Un...",37.371474,-121.967877,NaN
4,BRK-A,Berkshire Hathaway Inc.,Financial Services,7.762554e+11,"3555 Farnam Street, Omaha, NE, United States",41.257347,-95.965822,NaN
...,...,...,...,...,...,...,...,...
1310,WJRYY,West Japan Railway Company,Industrials,1.016414e+10,"4-24, Shibata 2-chome, Osaka, N/A, Japan",34.706983,135.496103,NaN
1311,SOUHY,South32 Limited,Basic Materials,1.014177e+10,"108 St Georges Terrace, Perth, WA, Australia",-31.954489,115.857179,NaN
1312,COVTY,Covestro AG,Basic Materials,1.006246e+10,"Kaiser-Wilhelm-Allee 60, Leverkusen, N/A, Germany",51.014248,6.985474,NaN
1313,BXP,"Boston Properties, Inc.",Real Estate,1.004093e+10,"Prudential Center, 800 Boylston Street, Boston...",42.348114,-71.079626,1.0


In [34]:
def create_ticker_map(info_df):
    us_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)
    
    district_colors = {}  # Dictionary to store colors for each district

    for index, row in info_df.iterrows():
        latitude = row['Latitude']
        longitude = row['Longitude']
        ticker = row['Ticker']
        headquarters = row['Headquarters']
        district = row['District']
        
        if pd.notnull(latitude) and pd.notnull(longitude):
            if district != 'N/A':
                if district not in district_colors:
                    district_colors[district] = 'blue' if len(district_colors) % 2 == 0 else 'green'
                
                color = district_colors[district]
            else:
                color = 'red'

            folium.Marker(
                location=[latitude, longitude],
                popup=f"{ticker} - {headquarters}<br>District: {district}",
                icon=folium.Icon(color=color, icon='times')
            ).add_to(us_map)
    
    return us_map

In [49]:
def create_us_map():
    us_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)
    # Add markers, layers, etc. to the map
    return us_map

In [50]:
def main():
    st.title("Ticker Headquarters Map")
    st.write("Displaying ticker headquarters on the map based on Federal Reserve districts.")
    st.write("Blue markers indicate known districts. Red markers indicate unknown districts.")

    # Read the saved HTML map
    us_map = create_us_map()


    # Display the map using Streamlit
    st.write(us_map._repr_html_(), unsafe_allow_html=True)

if __name__ == "__main__":
    main()

In [44]:
# 다운로드한 파일을 로컬 환경으로 가져오기
from google.colab import files
files.download('plot.py')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Streamlit app
st.title("Ticker Headquarters Map")
st.write("Displaying ticker headquarters on the map based on Federal Reserve districts.")
st.write("Blue markers indicate known districts. Red markers indicate unknown districts.")

# Display the map using Streamlit
st.markdown(us_map._repr_html_(), unsafe_allow_html=True)

In [ ]:
import streamlit as st

def main():
    st.title("Ticker Map")

    # Load the saved HTML map
    with open("ticker_map.html", "r") as f:
        map_html = f.read()

    # Display the map using Streamlit
    st.components.v1.html(map_html, width=800, height=600)

if __name__ == "__main__":
    main()
